# Balanceamento de classes
Feito em um notebook separado para carregar somente o mínimo de pacotes necessários para economia de memória

In [1]:
import numpy as np
from imblearn.over_sampling import SMOTE

In [2]:
X_train = np.loadtxt('data_files/X_train.csv', delimiter=',')

In [3]:
X_train.nbytes

1062644352

In [4]:
np.count_nonzero(X_train)

3568391

In [5]:
# Verificando esparsidade da matriz X_train

np.count_nonzero(X_train) / ((X_train.shape[0] * X_train.shape[1]))

0.026864235382488534

In [6]:
#X_train = X_train.astype('float16')

In [7]:
#X_train.nbytes

In [8]:
#X_train.shape

In [9]:
y_train = np.loadtxt('data_files/y_train.csv', delimiter=',')

In [10]:
y_train.shape

(2324,)

In [11]:
#y_train = y_train.astype('float16')

In [12]:
sm = SMOTE()

In [13]:
X_train_balanced, y_train_balanced = sm.fit_resample(X_train, y_train)

In [14]:
X_train_balanced.shape

(5877, 57156)

In [15]:
y_train_balanced.shape

(5877,)

In [16]:
from collections import Counter

## Conferindo a contagem das classes após a execução do SMOTE

In [17]:
Counter(y_train)

Counter({7.0: 653,
         6.0: 201,
         4.0: 481,
         2.0: 316,
         5.0: 165,
         1.0: 412,
         3.0: 56,
         9.0: 25,
         8.0: 15})

In [18]:
Counter(y_train_balanced)

Counter({7.0: 653,
         6.0: 653,
         4.0: 653,
         2.0: 653,
         5.0: 653,
         1.0: 653,
         3.0: 653,
         9.0: 653,
         8.0: 653})

## Salvando os resultados para a próxima etapa

In [19]:
#np.savetxt('data_files/X_train_balanced.csv', X_train_balanced, delimiter=',')
#np.savetxt('data_files/y_train_balanced.csv', y_train_balanced, delimiter=',')

In [20]:
# Reduzindo a precisão dos números para redução do espaço ocupado na memória
#np.savetxt('data_files/X_train_balanced_v5.csv', X_train_balanced, delimiter=',', fmt='%1.4e')
#np.savetxt('data_files/y_train_balanced_v5.csv', y_train_balanced, delimiter=',', fmt='%1.4e')

In [21]:
X_train_balanced.nbytes

2687246496

In [22]:
# Verificando esparsidade da matriz X_train

np.count_nonzero(X_train_balanced) / ((X_train_balanced.shape[0] * X_train_balanced.shape[1]))

0.041482265272623504

Matriz X_train_balanced ocupa 6,5 GB na memória e 3,5 GB em disco. Apresentou problemas de carregamento na máquina local para processamento posterior. Porém, a matriz apresenta apenas 4% de elementos não-nulos. Estratégia é gravar e carregar a matriz como matriz esparsa e depois convertê-la para array numpy.

## Tratamento como matriz esparsa

In [23]:
from scipy import sparse
from sys import getsizeof

In [24]:
X_train_blc_sparse = sparse.coo_matrix(X_train_balanced)

In [25]:
X_train_blc_sparse.shape

(5877, 57156)

In [26]:
getsizeof(X_train_balanced)

2687246608

In [27]:
getsizeof(X_train_blc_sparse)

48

In [28]:
sparse.save_npz('data_files/X_train_blc_sparse.npz', X_train_blc_sparse)

O tamanho do arquivo em disco foi reduzido para 91.9 MB

In [29]:
# Salvando o dataset de teste no mesmo formato para aplicação da redução de dimensionalidade
X_test = np.loadtxt('data_files/X_test.csv', delimiter=',')
X_test_sparse = sparse.coo_matrix(X_test)

In [31]:
sparse.save_npz('data_files/X_test_sparse.npz', X_test_sparse)

In [32]:
# Salvando o y_train_balanced, que será necessário na etapa de treinamento dos modelos de ML

In [33]:
np.savez('data_files/y_train_balanced.npz', y_train_balanced)